In [1]:
import sqlite3
import pandas as pd
import autog_student as ag

import json
from collections import defaultdict as dd

In [2]:
conn = sqlite3.connect('databases/nyc_health.db')
curs = conn.cursor()

In [3]:
# File name for your submission
# Just use the assignment name/number.  Blackboard will add your user name when I download them.
# Don't include underscores! It will mess up the pattern Blackboard uses to name files, and I won't know who you are.
file_name = 'Midterm.DATA302'
#Checkpoint2
# Name of the database - must be one directory up in the 'databases' folder
db_name = 'nyc_health.db'

# Do not change these:
PATH_SUB = 'submissions/' + file_name
PATH_DB  = 'databases/' + db_name

# Dictionary to store your submission
my_sql = {}

conn = sqlite3.connect(PATH_DB)
curs = conn.cursor()
pd.read_sql("SELECT * FROM sqlite_master;", conn)

,type,name,tbl_name,rootpage,sql
0,table,tRest,tRest,2,CREATE TABLE tRest (\n camis INTEGER PRIMAR...
1,table,tViol,tViol,3,CREATE TABLE tViol (\n viol_id TEXT PRIMARY...
2,index,sqlite_autoindex_tViol_1,tViol,4,None
3,table,tAction,tAction,5,CREATE TABLE tAction (\n action_id INTEGER ...
4,table,tInsp,tInsp,6,CREATE TABLE tInsp (\n camis INTEGER NOT NU...
5,index,sqlite_autoindex_tInsp_1,tInsp,7,None


In [4]:

sql = """
SELECT camis, dba, viol_id, viol_desc, count(viol_id) as NumViol
FROM tRest
JOIN tInsp USING(camis)
JOIN tViol USING(viol_id)
WHERE (viol_desc LIKE '%vermin%' or viol_desc LIKE '%mice%' or viol_desc LIKE '%roach%' or viol_desc LIKE '%rats%')
GROUP BY viol_id
ORDER BY NumViol DESC
;"""

my_sql['sql1'] = {}
my_sql['sql1']['sql'] = sql

sql2 = """
WITH 
TopViol as 
(
    SELECT cuisine, max(NumViol) as MaxViol, viol_id
    FROM (
        SELECT cuisine, viol_id, count(viol_id) as NumViol
        FROM tRest
        JOIN tInsp USING(camis)
        GROUP BY cuisine, viol_id
        )
    GROUP BY cuisine
)

SELECT cuisine, viol_id, viol_desc, MaxViol
FROM TopViol
JOIN tViol USING(viol_id)
ORDER BY cuisine ASC
;"""

my_sql['sql2'] = {}
my_sql['sql2']["sql"] = sql2

sql3 = """
SELECT action_desc, viol_desc, count(viol_id) as NumViol
FROM tInsp
JOIN tViol USING(viol_id)
JOIN tAction USING(action_id)
WHERE action_desc like '%closed%'
GROUP BY viol_id
ORDER BY NumViol DESC
LIMIT 5
;"""

my_sql['sql3'] = {}
my_sql['sql3']["sql"] = sql3

sql4 = """
WITH
MostClosed as
(
    SELECT camis, count(action_id) as NumAct
    FROM tInsp
    JOIN tAction USING(action_id)
    WHERE action_desc like '%closed%'
    GROUP BY camis
    ORDER BY NumAct DESC
    LIMIT 1
    
)

SELECT camis, dba, viol_desc, count(viol_id) as NumViol
FROM tRest
JOIN tInsp USING(camis)
JOIN tViol USING(viol_id)
WHERE camis = (SELECT camis FROM MostClosed)
GROUP BY viol_id
ORDER BY NumViol DESC
;"""

my_sql['sql4'] = {}
my_sql['sql4']["sql"] = sql4

sql5 = """
SELECT dba, count(DISTINCT camis) as NumLoc, count(viol_id) as NumViol, 1.*count(viol_id)/count(DISTINCT camis) as AvgViol 
FROM tRest
JOIN tInsp USING(camis)
JOIN tViol USING(viol_id)
GROUP BY dba
ORDER BY NumLoc DESC
;"""

my_sql['sql5'] = {}
my_sql['sql5']["sql"] = sql5

sql6 = """
WITH 
TotalCamis as 
(
    SELECT camis, dba, count(viol_id) as NumViol
    FROM tRest
    JOIN tInsp USING(camis)
    JOIN tAction USING(action_id)
    WHERE action_desc LIKE '%cited%' 
    GROUP BY dba
    HAVING NumViol > 1
)

SELECT camis, dba, viol_id, viol_desc, count(viol_id) as NumViol
FROM tRest
JOIN tInsp USING(camis)
JOIN tViol USING(viol_id)
JOIN tAction USING(action_id)
WHERE camis in (SELECT camis FROM TotalCamis)
GROUP BY viol_id
HAVING NumViol > 10
ORDER BY dba ASC
;"""

my_sql['sql6'] = {}
my_sql['sql6']["sql"] = sql6

sql7 = """
WITH 
SingleCami as 
(
    SELECT dba, count(DISTINCT camis) as DistCamis, count(viol_id) as NumViol
    FROM tRest
    JOIN tInsp USING(camis)
    JOIN tAction USING(action_id)
    WHERE action_desc LIKE '%cited%' 
    GROUP BY dba
    HAVING DistCamis = 1 AND NumViol > 10
),

TopViol as 
(
    SELECT dba, camis, max(NumViol) as MaxViol, viol_id
    FROM (
        SELECT camis, dba, viol_id, count(viol_id) as NumViol
        FROM tRest
        JOIN tInsp USING(camis)
        GROUP BY dba, viol_id
        )
    GROUP BY dba
)

SELECT camis, dba, viol_desc, MaxViol
FROM TopViol
JOIN tViol USING(viol_id)
JOIN SingleCami USING(dba)
ORDER BY MaxViol DESC
;"""

my_sql['sql7'] = {}
my_sql['sql7']["sql"] = sql7

sql8 = """
WITH
AllCombs as
(
    SELECT action_id, viol_id
    FROM tViol
    JOIN tAction
),
HasOccurred as
(
    SELECT action_id, viol_id, count(*) as NumViol
    FROM tInsp
    GROUP BY viol_id, action_id
)

SELECT count(*) as Total
FROM AllCombs
LEFT JOIN HasOccurred USING(action_id, viol_id)
WHERE NumViol IS NULL
;"""

my_sql['sql8'] = {}
my_sql['sql8']["sql"] = sql8

sql9 = """
WITH
AllCombs as
(
    SELECT action_id, viol_id
    FROM tViol
    JOIN tAction
),
HasOccurred as
(
    SELECT action_id, viol_id, count(*) as NumViol
    FROM tInsp
    GROUP BY viol_id, action_id
)

SELECT action_desc, viol_desc
FROM AllCombs
LEFT JOIN HasOccurred USING(action_id, viol_id)
JOIN tViol USING(viol_id)
JOIN tAction USING(action_id)
WHERE NumViol IS NULL
ORDER BY viol_desc ASC
;"""

my_sql['sql9'] = {}
my_sql['sql9']["sql"] = sql9

sql10 = """
WITH 
ViolCamis as 
(
    SELECT dba, count(viol_id) as NumViol
    FROM tRest
    JOIN tInsp USING(camis)
    JOIN tAction USING(action_id)
    WHERE action_desc LIKE '%cited%' 
    GROUP BY camis
    HAVING NumViol > 89
),

TopViol as 
(
    SELECT dba, camis, max(NumViol) as MaxViol, viol_id
    FROM (
        SELECT camis, dba, viol_id, count(viol_id) as NumViol
        FROM tRest
        JOIN tInsp USING(camis)
        GROUP BY dba, viol_id
        )
    GROUP BY dba
)

SELECT camis, dba, viol_id, viol_desc, MaxViol
FROM TopViol
JOIN tViol USING(viol_id)
JOIN ViolCamis USING(dba)
ORDER BY dba ASC
;"""

my_sql['sql10'] = {}
my_sql['sql10']["sql"] = sql10

my_sql

{'sql1': {'sql': "\nSELECT camis, dba, viol_id, viol_desc, count(viol_id) as NumViol\nFROM tRest\nJOIN tInsp USING(camis)\nJOIN tViol USING(viol_id)\nWHERE (viol_desc LIKE '%vermin%' or viol_desc LIKE '%mice%' or viol_desc LIKE '%roach%' or viol_desc LIKE '%rats%')\nGROUP BY viol_id\nORDER BY NumViol DESC\n;"},
 'sql2': {'sql': '\nWITH \nTopViol as \n(\n    SELECT cuisine, max(NumViol) as MaxViol, viol_id\n    FROM (\n        SELECT cuisine, viol_id, count(viol_id) as NumViol\n        FROM tRest\n        JOIN tInsp USING(camis)\n        GROUP BY cuisine, viol_id\n        )\n    GROUP BY cuisine\n)\n\nSELECT cuisine, viol_id, viol_desc, MaxViol\nFROM TopViol\nJOIN tViol USING(viol_id)\nORDER BY cuisine ASC\n;'},
 'sql3': {'sql': "\nSELECT action_desc, viol_desc, count(viol_id) as NumViol\nFROM tInsp\nJOIN tViol USING(viol_id)\nJOIN tAction USING(action_id)\nWHERE action_desc like '%closed%'\nGROUP BY viol_id\nORDER BY NumViol DESC\nLIMIT 5\n;"},
 'sql4': {'sql': "\nWITH\nMostClosed as\n

In [5]:
with open('Midterm.DATA302', 'w') as f:
    json.dump(my_sql, f)

In [6]:
with open(PATH_SUB, 'w') as f:
    json.dump(my_sql, f)

In [7]:
s = ag.Student('Midterm.DATA302', db_name)

In [8]:
s.CheckMySubmission()

Output saved to: output/me_2023-03-24-16-03-00-817131.DATA3024u 

===== RUNNING SETUP QUERIES =====

Note that the student version will run all action queries, however

some actions are disallowed during grading. See the documentation for more info.





===== DONE WITH SETUP QUERIES =====







-------------Submission for sql1 found-------------



SELECT camis, dba, viol_id, viol_desc, count(viol_id) as NumViol

FROM tRest

JOIN tInsp USING(camis)

JOIN tViol USING(viol_id)

WHERE (viol_desc LIKE '%vermin%' or viol_desc LIKE '%mice%' or viol_desc LIKE '%roach%' or viol_desc LIKE '%rats%')

GROUP BY viol_id

ORDER BY NumViol DESC

;



	Parameters: None

	Query executed w/o error: True



Output:



   camis              dba viol_id            viol_desc  NumViol

41374557    M NOODLE SHOP     08A Facility not vermin     39982

50003451        CHINA AAA     04L Evidence of mice or     27774

41516263       NO QUARTER     04M Live roaches present     9012

50058069 HENRI'S BACKYARD    

In [32]:
del s

In [34]:
conn.close()
